In [1]:
!pip3 install transformers datasets accelerate peft bitsandbytes torch evaluate tqdm numpy sentencepiece nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
!pip3 install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e9af2a8a19164ed1d284e356950574f17aa01a902956936b63ae7e9ea060721e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [4]:
!pip install -U datasets

In [5]:
import random
import numpy as np
import torch
import nltk
import time

from datasets import load_dataset
import evaluate

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    BitsAndBytesConfig # Important for QLoRA
)
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

2025-05-29 05:32:22.723488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748496742.918848      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748496742.977357      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# --- 1. Initial Configuration & Reproducibility ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [7]:
MODEL_NAME = "t5-small"
OUTPUT_DIR = "./t5_small_qlora_summarization_results" # Changed directory name to differentiate from LoRA
SAVED_MODEL_PATH = "./saved_models/t5_small_qlora_summarization" # Changed directory name

In [8]:
# --- 2. Quantization Configuration (BitsAndBytesConfig) for QLoRA ---
print("\n--- 4-bit Quantization Configuration (QLoRA) ---")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Enable 4-bit quantization
    bnb_4bit_quant_type="nf4", # Use NormalFloat 4-bit (NF4) format
    bnb_4bit_use_double_quant=True, # Enable double quantization for further memory savings
    # Choose dtype for computations. bfloat16 is better than float16 for precision if GPU supports it.
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16
)

# --- 3. Load Tokenizer and T5-small Model with Quantization ---
print(f"\n--- Loading tokenizer and base model {MODEL_NAME} (quantized) ---")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config, # Apply quantization configuration
    device_map="auto" # Automatically allocate parts of the model to GPUs (very important for quantization)
)

model = prepare_model_for_kbit_training(model)

# --- 4. Configure and Apply LoRA to the Quantized Model ---
print("\n--- Configuring and applying LoRA to the quantized model ---")
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q", "v","k", "o"], # Common modules in T5.
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Apply LoRA configuration to the quantized model
model = get_peft_model(model, lora_config)

print("Number of trainable parameters for the QLoRA model:")
model.print_trainable_parameters()

# model.to(device) # No need to call .to(device) anymore as device_map="auto" handles it


--- 4-bit Quantization Configuration (QLoRA) ---

--- Loading tokenizer and base model t5-small (quantized) ---


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


--- Configuring and applying LoRA to the quantized model ---
Number of trainable parameters for the QLoRA model:
trainable params: 589,824 || all params: 61,096,448 || trainable%: 0.9654


In [9]:
# --- 5. Prepare Data (No prefix used) ---
print("\n--- Loading and preprocessing CNN/DailyMail data (No prefix used) ---")
train_raw = load_dataset("cnn_dailymail", "3.0.0", split="train")
val_raw = load_dataset("cnn_dailymail", "3.0.0", split="validation")
test_raw = load_dataset("cnn_dailymail", "3.0.0", split="test")

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = examples["article"] # No "summarize: " prefix added
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(text_target=examples["highlights"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_ds = train_raw.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
tokenized_val_ds = val_raw.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])
tokenized_test_ds = test_raw.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])

print("Training set size:", len(tokenized_train_ds))
print("Validation set size:", len(tokenized_val_ds))
print("Test set size:", len(tokenized_test_ds))



--- Loading and preprocessing CNN/DailyMail data (No prefix used) ---


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Training set size: 287113
Validation set size: 13368
Test set size: 11490


In [10]:
# --- 6. Data Collator and Metrics ---
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # Use the quantized/LoRA model

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return result

# --- 7. Training Configuration (`Seq2SeqTrainingArguments`) ---
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_steps=2000,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    bf16=torch.cuda.is_available() and torch.cuda.is_bf16_supported(),
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    predict_with_generate=True,
    group_by_length=True,
)


# --- 8. Initialize Trainer and Train ---
print("\n--- Starting QLoRA training process ---")
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

start_time_train = time.time()
trainer.train()
end_time_train = time.time()
train_duration = end_time_train - start_time_train
print(f"\nQLoRA training completed in: {train_duration:.2f} seconds")



/tmp/ipykernel_19/2016289307.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



--- Starting QLoRA training process ---


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,2.031700,1.816780,25.050600,11.941800,20.537300,23.596700,19.999776


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.



QLoRA training completed in: 23513.27 seconds


In [11]:
# --- 9. Final Evaluation on Validation Set (best model) ---
print("\n--- Evaluating the best model on the Validation set ---")
final_metrics_qlora = trainer.evaluate(tokenized_val_ds) # Store QLoRA results in a separate variable
print("Final evaluation results on Validation (QLoRA):", final_metrics_qlora)

print("\n--- Evaluating the best model on the Test set ---")
final_metrics_qlora = trainer.evaluate(tokenized_test_ds) # Store QLoRA results in a separate variable
print("Final evaluation results on Test (QLoRA):", final_metrics_qlora)




--- Evaluating the best model on the Validation set ---


Final evaluation results on Validation (QLoRA): {'eval_loss': 1.817234992980957, 'eval_rouge1': 25.0528, 'eval_rouge2': 11.948, 'eval_rougeL': 20.5478, 'eval_rougeLsum': 23.5963, 'eval_gen_len': 19.999775583482943, 'eval_runtime': 809.0799, 'eval_samples_per_second': 16.522, 'eval_steps_per_second': 0.517, 'epoch': 0.9998885545525465}

--- Evaluating the best model on the Test set ---
Final evaluation results on Test (QLoRA): {'eval_loss': 1.830651044845581, 'eval_rouge1': 25.3761, 'eval_rouge2': 12.0497, 'eval_rougeL': 20.8101, 'eval_rougeLsum': 23.8781, 'eval_gen_len': 20.0, 'eval_runtime': 695.7939, 'eval_samples_per_second': 16.514, 'eval_steps_per_second': 0.517, 'epoch': 0.9998885545525465}


In [12]:
# --- 10. Save QLoRA adapter and Tokenizer ---
print(f"\n--- Saving QLoRA model and tokenizer at: {SAVED_MODEL_PATH} ---")
trainer.save_model(SAVED_MODEL_PATH)
tokenizer.save_pretrained(SAVED_MODEL_PATH)
print("Model saving complete.")

# --- 11. Example of loading and using the trained model (Inference) ---
print("\n--- Example of generating a summary after training (QLoRA) ---")
from peft import PeftModel, PeftConfig

peft_model_id = SAVED_MODEL_PATH
config = PeftConfig.from_pretrained(peft_model_id)

# Reload the base model with quantization configuration
base_model_for_inference = AutoModelForSeq2SeqLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config, # Very important: must re-apply quantization config
    device_map="auto"
)

# Load LoRA adapter and combine with the quantized base model
inference_model = PeftModel.from_pretrained(base_model_for_inference, peft_model_id)
# inference_model.to(device) # No need to call .to(device) as device_map="auto" handles it
inference_model.eval()

sample_article = val_raw["article"][5]
print(f"\nOriginal article (first 500 characters):\n{sample_article[:500]}...")

input_text_for_inference = sample_article
inputs_for_inference = tokenizer(
    input_text_for_inference,
    return_tensors="pt",
    max_length=max_input_length,
    truncation=True
)

with torch.no_grad():
    summary_ids = inference_model.generate(
        input_ids=inputs_for_inference["input_ids"].to(device), # Fix TypeError
        attention_mask=inputs_for_inference["attention_mask"].to(device),
        max_new_tokens=max_target_length,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True,
        no_repeat_ngram_size=3
    )

generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(f"\nGenerated summary:\n{generated_summary}")
print(f"\nOriginal summary:\n{val_raw['highlights'][5]}")


--- Saving QLoRA model and tokenizer at: ./saved_models/t5_small_qlora_summarization ---
Model saving complete.

--- Example of generating a summary after training (QLoRA) ---

Original article (first 500 characters):
(CNN)My vote for Father of the Year goes to Curt Schilling. The former Major League Baseball pitcher recently fired off a series of fastballs and mowed down a group of Twitter trolls who made the mistake of tweeting vulgar and sexually-explicit comments about Schilling's teenage daughter. The drama started, innocently enough, on February 25, when Schilling played the role of a proud father. He sent a tweet congratulating his daughter, Gabby, on being accepted to Salve Regina University, where sh...

Generated summary:
Curt Schilling mowed down a group of Twitter trolls who made the mistake of tweeting vulgar and sexually-explicit comments about his daughter. Schilling sent a tweet congratulating his daughter, Gabby, on being accepted to Salve Regina University.

Original